Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [798]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from random import random, choice, randint
from functools import reduce
from collections import namedtuple
from copy import copy
from prettytable import PrettyTable
import time
import math

In [799]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = np.array([
            [random() < density for i in range(num_points)]
            for j in range(num_sets)
        ])
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [800]:
NUM_POINTS = [100, 1000,5000]
NUM_SETS = NUM_POINTS
DENSITY = 0.3

In [801]:
def valid_points(sets, state):
    return  np.any(sets[state, :], axis=0).sum()

In [802]:
def is_valid(state,sets):
    return np.all(np.any(sets[state, :], axis=0))

In [803]:

def fitness1(sets, problem_size, state):
    def __name__():
        return 'fitness1'

    cost = sum(state)
    valid = valid_points(sets, state)
    return valid, -cost

def fitness2(sets, problem_size, state):
    def __name__():
        return 'fitness2'

    cost = sum(state)
    valid = np.all(np.any(sets[state, :], axis=0))
    return valid, -cost


def fitness3(sets, problem_size, state):
    def __name__():
        return 'fitness3'

    valid = valid_points(sets, state)
    return valid


def fitness4(sets, problem_size, state):
    def __name__():
        return 'fitness4'

    div=  5
    valid = math.floor(np.any(sets[state, :], axis=0).sum()/div)
    return valid

def fitness5(sets, problem_size, state):
    def __name__():
        return 'fitness5'

    cost = sum(state)
    total=sets.sum(axis=0)
    r=np.any(sets[state, :], axis=0)
    weighted=(r*(1/total))
    return weighted.sum(),-cost


def fitness6(sets, problem_size, state):
    def __name__():
        return 'fitness5'

    cost = sum(state)
    total=sets.sum(axis=0)
    r=np.any(sets[state, :], axis=0)
    weighted=(r*(1/total))
    return weighted.sum()

In [804]:
def tweak(problem_size, state, sets, method="stochastic"):
    new_state = copy(state)
    if method == "stochastic" :
        index = randint(0, problem_size - 1)
        new_state[index] = not new_state[index]
        return new_state
    elif method == "steepest_ascent" :
        best_state = None
        best_cost = -1
        for i in range(len(state)):
            new_state=copy(state)
            new_state[i] = not new_state[i]
            new_cost = valid_points(sets, new_state)
            if new_cost > best_cost:
                best_state = new_state
                best_cost = new_cost
        return best_state
    elif method == "first_choice" :
        for i in range(len(state)):
            if state[i]==False:
                new_state[i] = not new_state[i]
                return new_state
    return new_state

In [805]:
def hill_climbing(fitness,problem_size,current_state,sets,method):
    current_cost = fitness(sets,problem_size,current_state)
    count=0
    while True:
        new_state = tweak(problem_size,current_state,sets,method)
        new_cost = fitness(sets,problem_size,new_state)
        count+=1
        if new_cost >= current_cost:
            current_state = new_state
            current_cost = new_cost
        else:
            return current_state, current_cost, count

In [806]:
def restart_hill_climbing(fitness,problem_size,current_state,sets,method, max_iterations=100):
    count=0
    current_cost = fitness(sets,problem_size,current_state)
    while not is_valid(current_state,sets) and count < max_iterations:
        current_state, current_cost, current_count = hill_climbing(fitness,problem_size,current_state,sets,method)
        count+=current_count
    return current_state, current_cost, count

In [807]:
def iterated_local_search(fitness,problem_size,current_state,sets,method, max_iterations=100):
    count=0
    best_state=copy(current_state)
    best_cost = fitness(sets,problem_size,best_state)
    while not is_valid(best_state,sets) and count < max_iterations:
        new_state, new_cost, new_count = hill_climbing(fitness,problem_size,best_state,sets,method)
        count+=new_count
        if new_cost >= best_cost:
            best_state = new_state
            best_cost = new_cost
    return best_state, best_cost, count

In [808]:
def temp_schedule(T, iter, criteria="cauchy"):
    if criteria=="boltzmann":
        return T/(1+iter)
    elif criteria=="cauchy":
        return T/(1+math.log(iter))
    elif criteria=="exp":
        return T*math.exp(-iter)
    
def simulated_annealing(fitness,problem_size,initial_state,sets,method, max_iteration=100, p=None):
    T=problem_size/5
    count=0
    current_state=copy(initial_state)
    current_cost=fitness(sets,problem_size,current_state)
    while not is_valid(current_state,sets) and count<max_iteration:
        next_state=tweak(problem_size,current_state,sets,method)
        next_cost=fitness(sets,problem_size,next_state)
        count+=1
        error=next_cost-current_cost
        if p is None:
            if random()>=math.exp(-error/T):
                    current_state=next_state 
        else:
            if p>=math.exp(-error/T):
                current_state=next_state
        T=temp_schedule(T, count)
    return current_state, len(current_state), count
    

In [809]:
def compare(num_points, num_sets, density, fitness_functions, methods, variants):
    for i in range(len(num_points)):
        t = PrettyTable(["Points", "Sets", "Fitness Function", "Method","Variant","Count","Time","Cost","is_valid"])
        problem_size=num_points[i]
        set_num=num_sets[i]
        sets = make_set_covering_problem(problem_size, set_num, density)
        for fitness in fitness_functions:
            for method in methods:
                if method=="hill_climbing":
                    algorithm=hill_climbing
                elif method=="simulated_annealing":
                    algorithm=simulated_annealing
                elif method=="restart_hill_climbing":
                    algorithm=restart_hill_climbing
                elif method=="iterated_local_search":
                    algorithm=iterated_local_search
                for variant in variants:
                    start=time.time()
                    current_state=[False]*num_sets[i]
                    best_state, best_cost, count = algorithm(fitness,problem_size,current_state,sets,variant)
                    end=time.time()
                    t.add_row(
                            [
                                str(problem_size),
                                str(set_num),
                                str(fitness.__name__),
                                method,
                                variant,
                                str(count),
                                "{:.3e}".format(end - start),
                                str(best_cost),
                                is_valid(best_state,sets)
                            ]
                    )
        print(t)  

In [810]:
compare(NUM_POINTS, NUM_SETS, DENSITY, fitness_functions=[fitness1, fitness5],methods=["hill_climbing","restart_hill_climbing","iterated_local_search"],variants=["stochastic","steepest_ascent","first_choice"])

+--------+------+------------------+-----------------------+-----------------+-------+-----------+---------------------------+----------+
| Points | Sets | Fitness Function |         Method        |     Variant     | Count |    Time   |            Cost           | is_valid |
+--------+------+------------------+-----------------------+-----------------+-------+-----------+---------------------------+----------+
|  100   | 100  |     fitness1     |     hill_climbing     |    stochastic   |   11  | 1.919e-04 |         (99, -10)         |  False   |
|  100   | 100  |     fitness1     |     hill_climbing     | steepest_ascent |   7   | 7.377e-03 |         (100, -6)         |   True   |
|  100   | 100  |     fitness1     |     hill_climbing     |   first_choice  |   12  | 1.519e-04 |         (98, -11)         |  False   |
|  100   | 100  |     fitness1     | restart_hill_climbing |    stochastic   |   15  | 2.060e-04 |         (100, -12)        |   True   |
|  100   | 100  |     fitness1    

In [811]:
compare(NUM_POINTS, NUM_SETS, DENSITY, fitness_functions=[fitness3, fitness6],methods=["simulated_annealing"],variants=["stochastic","steepest_ascent","first_choice"])

+--------+------+------------------+---------------------+-----------------+-------+-----------+------+----------+
| Points | Sets | Fitness Function |        Method       |     Variant     | Count |    Time   | Cost | is_valid |
+--------+------+------------------+---------------------+-----------------+-------+-----------+------+----------+
|  100   | 100  |     fitness3     | simulated_annealing |    stochastic   |   10  | 5.190e-04 | 100  |   True   |
|  100   | 100  |     fitness3     | simulated_annealing | steepest_ascent |   6   | 6.764e-03 | 100  |   True   |
|  100   | 100  |     fitness3     | simulated_annealing |   first_choice  |   21  | 4.773e-04 | 100  |   True   |
|  100   | 100  |     fitness6     | simulated_annealing |    stochastic   |   18  | 6.189e-04 | 100  |   True   |
|  100   | 100  |     fitness6     | simulated_annealing | steepest_ascent |   11  | 8.220e-03 | 100  |   True   |
|  100   | 100  |     fitness6     | simulated_annealing |   first_choice  |   2